In [ ]:
!apt-get update
!apt-get install -y openjdk-11-jdk

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 4s (72.8 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to pr

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install jpype1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 14.6 MB/s eta 0:00:00


In [4]:
import jpype
import jpype.imports
from jpype.types import *

# JVM'yi başlat
jpype.startJVM('-ea', '-Djava.class.path=/content/drive/MyDrive/zemberek-full.jar') # DRIVE'A ZEMBEREK İNDİR

In [5]:
from jpype import JClass, JString, startJVM, getDefaultJVMPath, shutdownJVM

TurkishMorphology = JClass('zemberek.morphology.TurkishMorphology')
TurkishTokenizer = JClass('zemberek.tokenization.TurkishTokenizer')
morphology = TurkishMorphology.createWithDefaults()
sentence_tokenizer = TurkishTokenizer.DEFAULT

In [6]:
def pos_tagging(text):
    analysis_results = morphology.analyzeSentence(JString(text))
    pos_tags = []
    for result in analysis_results:
        for analysis in result.getAnalysisResults():
            pos_tags.append(str(analysis.getPos().shortForm))
    return pos_tags

def stem_word(word):

    stems = ''
    analysis = morphology.analyzeSentence(JString(word))
    best = morphology.disambiguate(JString(word), analysis).bestAnalysis()

    for parse in best:
        stems = parse.getLemmas()[0]

    return stems

In [11]:
import pandas as pd
df = pd.read_csv('companies.csv')
df.head()

,Content,company
0,apple tarafından yılın uygulaması seçilmiştir,apple
1,facebook bu ağı kendine entegre edicek ha gere...,facebook
2,facebookun satın almasıyla şüphesiz eskisinden...,facebook
3,facebook tarafından 1 milyar dolara satın alın...,facebook
4,googleın digg kurucusu efsane başgan kevin ros...,instagram


In [12]:
company_names = df['company'].str.lower().unique().tolist()
print(company_names)

['apple', 'facebook', 'instagram', 'google', 'ttnet', 'youtube', 'starbucks', 'twitter', 'steam', 'ınstagram', 'telekom', 'microsoft', 'ekşi', 'türknet', 'hepsiburada', 'teknosa', 'sabancı', 'mediamarkt', 'vatan', 'vatan bilgisayar', 'media markt', 'bimeks', 'logitech', 'superonline', 'turkcell', 'ups', 'bookingcom', 'turknet', 'isp', 'turknetin', 'btk', 'turknete', 'vodafone', 'turktelekom', 'tt', 'türktelekom', 'netflix', 'iss', 'digiturk', 'turkcell superonline', 'avea', 'xbox', 'ubisoft', 'blizzard', 'twitch', 'kick', 'discord', 'tiktok', 'x', 'türkcell', 'türkcellhizmet', 'türkcelldestek', 'süperonline', 'intel', 'amd', 'nvidia']


In [40]:
import string

def is_punctuation_only(s):
    return all(char in string.punctuation for char in s)

conjuctions = ['ve', 'ama', 'fakat', 'ancak', 'çünkü', 'lakin', 'sonra', 'bu nedenle', 'bu yüzden', 'acaba']


def find_syntagma(word):
    pos = pos_tagging(word)
    if not isinstance(word, str):
            word = str(word)

    if word != stem_word(word):
        if (word.endswith('i') or word.endswith('ı')) and (not word.endswith('di') or not word.endswith('dı')):
            return 'BN'  # Belirtili Nesne
        elif word.endswith('e') or word.endswith('a') or word.endswith('den') or word.endswith('dan'):
            return 'DT'  # Dolaylı Tümleç
        elif word.endswith('le') or word.endswith('la'):
            return 'I'  # Araç Tümleci
    return None

def find_completion(word1, word2):
    words = []

    if not isinstance(word1, str):
        words.append(str(word1))
    else:
        words.append(word1)
    if not isinstance(word2, str):
        words.append(str(word2))
    else:
        words.append(word2)


    check = 0
    if words[0].endswith('in') or words[0].endswith('ın') or words[0].endswith('un') or words[0].endswith('ün'):
         check += 1

    if words[1].endswith('e') or words[1].endswith('a'):
         check += 1

    if check == 2:
         return True

    return False

def contains_verb(tokens, is_completion=False):
    for token in tokens:
        if not isinstance(token, str):
            token = str(token)
        analysis = morphology.analyze(JString(token))
        for result in analysis:
            if result.getPos().getStringForm() == "Verb":
              if is_completion == False:
                return True
              else:
                if not find_completion(tokens[0], tokens[1]):
                  return True
    return False

def contains_noun_or_adj(tokens, posit=False):
    i = 0
    for token in tokens:
      if find_syntagma(token) is None:
        analysis = morphology.analyze(JString(token))
        for result in analysis:
            if result.getPos().getStringForm() in ["Noun", "Adj"]:
                if posit:
                    return i
                return True
        i += 1
    return False

def contains_company_names(tokens):
  i = 0

  tokens = tokens.split()

  for company_name in company_names:
    for token in tokens:
      if token.startswith(company_name):
        return True


def tokenize_sentence(text):
    if not isinstance(text, str):
        raise TypeError("Input text must be a string")

    tokens = sentence_tokenizer.tokenizeToStrings(JString(text))
    sentences = []
    current_sentence = []
    completions = []
    posis = []
    start_of_current_sentence = 0

    for i, token in enumerate(tokens):
        token_text = str(token)
        current_sentence.append(token_text)

        if len(current_sentence) >= 2 and find_completion(tokens[i-1], tokens[i]):
            completions.append(''.join(current_sentence[-2] + ' ' + current_sentence[-1]))
            posis.append(int(len(current_sentence) - 2))
            current_sentence.pop()
            current_sentence.pop()
            start_of_current_sentence = i + 1
            continue

        if token_text in ['.', '!', '?']:
            for completion, pos in zip(completions, posis):
                    current_sentence.insert(pos, completion)

            completions.clear()
            posis.clear()
            sentences.append(' '.join(current_sentence).strip())
            current_sentence = []
            start_of_current_sentence = i + 1
        elif token_text in conjuctions:
            if (contains_verb(current_sentence) and \
                contains_verb(tokens[i+1:])) or len(tokens[i+1:]) == 0:
                for completion, pos in zip(completions, posis):
                    current_sentence.insert(pos, completion)

                completions.clear()
                posis.clear()
                sentences.append(' '.join(current_sentence[:-1]).strip())
                current_sentence = [token_text]
                start_of_current_sentence = i
            elif i > 0 and (contains_verb([tokens[i-1]]) and contains_noun_or_adj(tokens[i+1:])) or (contains_verb([tokens[i+1:]]) and contains_noun_or_adj(tokens[i-1])):
                sentences.append(' '.join(current_sentence[:-1]).strip())
                current_sentence = [token_text]
                start_of_current_sentence = i
        elif i != len(tokens) - 1:
            if len(current_sentence) > 1 and contains_verb(current_sentence) and contains_noun_or_adj(current_sentence) and \
                find_syntagma(tokens[i+1]) is None and tokens[i+1] not in conjuctions:
                    for completion, pos in zip(completions, posis):
                        current_sentence.insert(pos, completion)

                    completions.clear()
                    posis.clear()
                    sentences.append(' '.join(current_sentence).strip())
                    current_sentence = []
                    start_of_current_sentence = i + 1


    if current_sentence:
        for completion, pos in zip(completions, posis):
                  current_sentence.insert(pos, completion)

        completions.clear()
        posis.clear()
        sentences.append(' '.join(current_sentence).strip())

    return sentences


texts = ['superonline internet bağlattık çok kötü çıktı onun yerine vodafone daha iyi bir seçenek gibi']


sentences = []
sentences = tokenize_sentence(texts[0])

print(sentences)


['superonline internet bağlattık', 'çok kötü çıktı', 'onun yerine vodafone daha iyi bir seçenek gibi']


In [41]:
def merge_texts(tokens):
  merge_sentences = []
  merge_sentence = []
  company_count = 0
  for token in tokens:
    if contains_company_names(token):
      company_count += 1

    if not token.endswith('.') and not sentence.endswith('!') and not token.endswith('?'):
      if company_count < 2 and merge_sentence:
        merge_sentence.append(token)
      elif company_count == 1 and not merge_sentence:
        merge_sentence.append(token)
      elif company_count > 1:
        merge_sentences.append(' '.join(merge_sentence))
        merge_sentence = [token]
        company_count = 0
        if contains_company_names(token):
          company_count = 1
      elif company_count < 2 and not merge_sentence:
        merge_sentence.append(token)

    else:
      merge_sentences.append(token)

  if merge_sentence:
    merge_sentences.append(' '.join(merge_sentence))
    merge_sentence.clear()

  return merge_sentences


In [42]:
new_sentences = []
new_sentences.append(merge_texts(sentences))

print(new_sentences)

[['superonline internet bağlattık çok kötü çıktı', 'onun yerine vodafone daha iyi bir seçenek gibi']]


In [38]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline, AutoModelForTokenClassification

sentiment_model = AutoModelForSequenceClassification.from_pretrained("ikball/turkish-sentiment", from_tf=True)
sentiment_tokenizer = AutoTokenizer.from_pretrained("ikball/turkish-sentiment")
sentiment_pipeline = pipeline("sentiment-analysis", tokenizer=sentiment_tokenizer, model=sentiment_model)

KeyboardInterrupt: 

In [ ]:
ner_model = AutoModelForTokenClassification.from_pretrained("mext741/turkish-ner-company")
ner_tokenizer = AutoTokenizer.from_pretrained("mext741/turkish-ner-company")
ner_pipeline = pipeline("ner", tokenizer=ner_tokenizer, model=ner_model)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
result_list = []

i = 1
for i, sentences in enumerate(new_sentences_list, start=1):
    result_dict = {
        'id': i,
        'results': {
            "sentences": [],
            "ner_results": [],
            "sentiment_scores": []
        }
    }

    for sentence in sentences:
        ner_results = ner_pipeline(sentence)
        sentiment_results = sentiment_pipeline(sentence)

        result_dict["results"]["sentences"].append(sentence)
        if len(ner_results) > 0:
            result_dict["results"]["ner_results"].append(ner_results)
        if len(sentiment_results) > 0:
            result_dict["results"]["sentiment_scores"].append(sentiment_results)

    result_list.append(result_dict)

In [ ]:
import json

processed_output_list = []

for result_dict in result_list:
    processed_output = {
        'id': result_dict['id'],
        'sentences': result_dict["results"]['sentences'],
        'ner_results': [
            [{'entity': entity['entity'], 'word': entity['word']} for entity in sentence_ner]
            for sentence_ner in result_dict["results"]['ner_results']
        ],
        'sentiment_scores': [
            [{'label': sentiment['label']} for sentiment in sentence_sentiment]
            for sentence_sentiment in result_dict["results"]['sentiment_scores']
        ]
    }

    processed_output_list.append(processed_output)

with open('output.json', 'w', encoding='utf-8') as json_file:
    json.dump(processed_output_list, json_file, ensure_ascii=False, indent=4)

print("JSON dosyası başarıyla oluşturuldu.")

JSON dosyası başarıyla oluşturuldu.
